# Editor

In [1]:
import os
import subprocess
import webvtt
import google.generativeai as genai
import json
import sys
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, CompositeVideoClip
from io import StringIO
import requests
import shutil # <-- ¡NUEVO IMPORT! Necesario para copiar tu video
from dotenv import load_dotenv

load_dotenv()

# --- FUNCIONES DE TIEMPO (SIN CAMBIOS) ---
def to_seconds(time_str):
    """Convierte un string de tiempo 'HH:MM:SS' o 'MM:SS' a segundos."""
    try:
        time_str = time_str.split('.')[0]
        parts = list(map(int, time_str.split(':')))
        if len(parts) == 3:  # HH:MM:SS
            h, m, s = parts
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:  # MM:SS
            m, s = parts
            return m * 60 + s
        else:
            raise ValueError("Formato de tiempo no válido")
    except ValueError as e:
        print(f"Error al convertir tiempo '{time_str}': {e}", file=sys.stderr)
        return 0

def from_seconds(total_seconds):
    """Convierte segundos a un string de tiempo 'HH:MM:SS'."""
    h = int(total_seconds // 3600)
    m = int((total_seconds % 3600) // 60)
    s = int(total_seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

# --- FUNCIÓN UTILITARIA PARA DESCARGAR FUENTE DE EMOJIS (SIN CAMBIOS) ---
def descargar_fuente_emoji(font_path="NotoColorEmoji.ttf"):
    """
    Descarga la fuente Noto Color Emoji si no existe,
    necesaria para que moviepy renderice emojis.
    """
    if os.path.exists(font_path):
        print(f"Fuente de emoji encontrada: {font_path}")
        return font_path

    print("Descargando fuente de emoji (NotoColorEmoji.ttf)...")
    url = "https://github.com/googlefonts/noto-emoji/raw/main/fonts/NotoColorEmoji.ttf"
    try:
        r = requests.get(url, allow_redirects=True)
        r.raise_for_status()
        with open(font_path, 'wb') as f:
            f.write(r.content)
        print("Fuente de emoji descargada exitosamente.")
        return font_path
    except Exception as e:
        print(f"Error descargando la fuente de emoji: {e}", file=sys.stderr)
        print("Los emojis podrían no renderizarse correctamente.", file=sys.stderr)
        return None # Retorna None si falla

# --- FUNCIÓN DE SUBTÍTULOS (MODIFICADA PARA LEER ARCHIVO LOCAL) ---
def leer_subtitulos_locales(vtt_path): # <-- ¡MODIFICADO! Cambia el argumento
    """
    Lee un archivo .vtt local y lo devuelve
    como un texto formateado Y como un objeto webvtt.
    """
    captions = None
    transcript = ""
    try:
        # <-- ¡TODA ESTA SECCIÓN FUE MODIFICADA! ---
        print(f"Leyendo archivo de subtítulos local: {vtt_path}...")
        if not os.path.exists(vtt_path):
            raise FileNotFoundError(f"No se pudo encontrar el archivo de subtítulos en: {vtt_path}")

        with open(vtt_path, 'r', encoding='utf-8') as f:
            vtt_content = f.read()
        # --- FIN DE LA MODIFICACIÓN ---

        captions = webvtt.read_buffer(StringIO(vtt_content))
        for caption in captions:
            timestamp = caption.start
            timestamp_clean = timestamp.split('.')[0]
            transcript += f"[{timestamp_clean}] {caption.text.strip().replace(chr(10), ' ')}\n"
        
        print(f"Subtítulos leídos y formateados exitosamente.")
        return transcript, captions
    except Exception as e:
        print(f"Error leyendo o procesando el archivo VTT: {e}", file=sys.stderr)
        return None, None
    # No hay 'finally' para borrar, ya que es el archivo original del usuario

# --- FUNCIÓN DE IA (CORTES) (SIN CAMBIOS) ---
def obtener_cortes_para_eliminar(api_key, transcript, duracion_minima_seg=0.1):
    print(f"Conectando con Gemini para analizar relleno verbal (duración > {duracion_minima_seg}s)...")
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-2.0-flash')
        prompt = f"""
        Eres un editor de video cuya tarea es eliminar "relleno" obvio
        de una transcripción.

        Analiza la siguiente transcripción e identifica segmentos para ELIMINAR
        basado en dos criterios. Sé conservador.

        CRITERIOS PARA ELIMINAR:
        1.  **Muletillas / Rellenos:** Palabras o sonidos como 'ehh', 'mmm',
            'pues...', 'esteee...', 'o sea...', 'bueno...'.
            Tu objetivo es eliminar las que *interrumpen la fluidez*.
        2.  **Reinicios Falsos CLAROS:** Oraciones que el orador
            empieza, abandona a mitad de frase, y vuelve a empezar
            (ej. "Y el video... no, mejor dicho, el audio...")

        REGLAS CRÍTICAS:
        -   **NO ELIMINES NADA POR 'REPETICIÓN'.**
        -   **SÉ CONSERVADOR.** Si dudas, NO LO CORTES.
        -   Si el video está limpio, devuelve una lista vacía [].

        Tu respuesta DEBE ser únicamente un objeto JSON:
        - "motivo": ("muletilla", "reinicio falso")
        - "inicio": El timestamp 'HH:MM:SS' de inicio.
        - "fin": El timestamp 'HH:MM:SS' de fin.

        Ejemplo de respuesta:
        [
          {{"motivo": "muletilla", "inicio": "00:02:15", "fin": "00:02:17"}}
        ]

        Aquí está la transcripción:
        ---
        {transcript}
        ---
        """
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        print("Análisis de IA completado. Parseando y filtrando segmentos...")
        secciones = json.loads(cleaned_response)

        secciones_filtradas = []
        for s in secciones:
            try:
                inicio_seg = to_seconds(s['inicio'])
                fin_seg = to_seconds(s['fin'])
                duracion_corte = fin_seg - inicio_seg
                if duracion_corte > 0 and duracion_corte >= duracion_minima_seg:
                    secciones_filtradas.append(s)
                else:
                    print(f"  · Descartando corte de IA (duración inválida o muy corta): {s}")
            except Exception:
                pass # Ignorar cortes malformados

        print(f"Se identificaron {len(secciones_filtradas)} cortes de IA válidos (duración >= {duracion_minima_seg}s).")
        return secciones_filtradas
    except Exception as e:
        print(f"Error al contactar con Gemini o parsear JSON: {e}", file=sys.stderr)
        return []

# --- FUNCIÓN DE IA PARA EMOJIS (SIN CAMBIOS) ---
def obtener_emojis_para_subtitulos(api_key, captions_obj, secciones_para_eliminar):
    """
    Filtra los subtítulos buenos y pide a Gemini que sugiera emojis para ellos.
    """
    print("Conectando con Gemini para analizar y sugerir emojis...")
    if not captions_obj:
        return []

    # 1. Pre-procesar las secciones a eliminar para una búsqueda rápida
    cortes_seg = []
    for corte in secciones_para_eliminar:
        try:
            cortes_seg.append((to_seconds(corte['inicio']), to_seconds(corte['fin'])))
        except Exception:
            continue

    # 2. Filtrar solo los subtítulos que NO están en una sección eliminada
    captions_buenos = []
    transcript_buena = ""
    for cap in captions_obj:
        es_bueno = True
        for inicio, fin in cortes_seg:
            if cap.start_in_seconds >= inicio and cap.start_in_seconds < fin:
                es_bueno = False
                break
        if es_bueno:
            captions_buenos.append(cap)
            timestamp_clean = cap.start.split('.')[0]
            transcript_buena += f"[{timestamp_clean}] {cap.text.strip().replace(chr(10), ' ')}\n"

    if not transcript_buena:
        print("No se encontró transcripción válida después del filtrado.")
        return []

    # 3. Enviar la transcripción "buena" a Gemini
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-2.0-flash')
        prompt = f"""
        Eres un editor de video creativo. Analiza la siguiente transcripción
        e identifica palabras o frases clave que puedan ser realzadas
        con un emoji relevante.

        REGLAS:
        1.  Sé selectivo. No añadas emojis a cada línea, solo a
            conceptos visuales fuertes (ej. idea 💡, dinero 💰,
            rápido 🚀, mundo 🌍, amor ❤️, éxito 🏆).
        2.  El emoji debe aparecer durante la palabra clave.
        3.  El "fin" debe ser 1 o 2 segundos después del "inicio".
        4.  Usa los timestamps originales del texto.

        Tu respuesta DEBE ser únicamente un objeto JSON (una lista de objetos):
        - "emoji": El emoji unicode (ej. "💡").
        - "inicio": El timestamp 'HH:MM:SS' de inicio.
        - "fin": El timestamp 'HH:MM:SS' de fin.

        Ejemplo de respuesta:
        [
          {{"emoji": "💡", "inicio": "00:02:15", "fin": "00:02:17"}},
          {{"emoji": "💰", "inicio": "00:05:31", "fin": "00:05:33"}}
        ]

        Aquí está la transcripción:
        ---
        {transcript_buena}
        ---
        """
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        print("Análisis de emojis completado.")
        return json.loads(cleaned_response)
    except Exception as e:
        print(f"Error al contactar con Gemini para emojis: {e}", file=sys.stderr)
        return []

# --- FUNCIÓN DE SILENCIOS (SIN CAMBIOS) ---
def identificar_silencios_largos(captions_obj, umbral_segundos=2.0):
    print(f"Identificando silencios de más de {umbral_segundos} segundos...")
    if not captions_obj:
        return []
    silencios = []
    sorted_captions = sorted(captions_obj, key=lambda c: c.start_in_seconds)
    for i in range(len(sorted_captions) - 1):
        cap_actual = sorted_captions[i]
        cap_siguiente = sorted_captions[i+1]
        gap = cap_siguiente.start_in_seconds - cap_actual.end_in_seconds
        if gap >= umbral_segundos:
            print(f"  · Silencio largo detectado: {from_seconds(cap_actual.end_in_seconds)} a {from_seconds(cap_siguiente.start_in_seconds)}")
            silencios.append({
                "motivo": "silencio largo",
                "inicio": from_seconds(cap_actual.end_in_seconds),
                "fin": from_seconds(cap_siguiente.start_in_seconds)
            })
    return silencios

# --- FUNCIÓN DE ENSAMBLAJE (MODIFICADA PARA USAR ARCHIVO LOCAL) ---
def ensamblar_video_editado(video_local_path, secciones_para_eliminar): # <-- ¡MODIFICADO!
    """
    Usa un video local y lo ensambla.
    Devuelve la RUTA al video final si tiene éxito.
    """
    video_original_path = "video_original_para_editar.mp4"
    video_final_path = "video_final_editado.mp4" # Video base cortado
    try:
        
        # <-- ¡TODA ESTA SECCIÓN FUE MODIFICADA! ---
        # Se eliminó la descarga de yt-dlp.
        print(f"Usando video local: {video_local_path}")
        if not os.path.exists(video_local_path):
            raise FileNotFoundError(f"No se pudo encontrar el archivo de video en: {video_local_path}")
        
        # Copiar el video a la ruta de trabajo que el script espera
        print(f"Copiando video a la ruta de trabajo temporal: {video_original_path}...")
        shutil.copy(video_local_path, video_original_path)
        print("Copia completada.")
        # --- FIN DE LA MODIFICACIÓN ---


        print("Video listo para procesar.")
        video = VideoFileClip(video_original_path)
        video_duration = video.duration
        if not secciones_para_eliminar:
            print("No se especificaron cortes. El video no será modificado.")
            video.close()
            # Renombramos el archivo copiado para que sirva como "editado"
            if os.path.exists(video_original_path):
                os.rename(video_original_path, video_final_path)
            return video_final_path # Devuelve la ruta

        # 1. Ordenar y fusionar cortes (Lógica sin cambios)
        cortes_ordenados = sorted(secciones_para_eliminar, key=lambda x: to_seconds(x['inicio']))
        cortes_fusionados = []
        if cortes_ordenados:
            current_merged_corte = cortes_ordenados[0]
            for i in range(1, len(cortes_ordenados)):
                next_corte = cortes_ordenados[i]
                current_fin_seg = to_seconds(current_merged_corte['fin'])
                next_inicio_seg = to_seconds(next_corte['inicio'])
                if next_inicio_seg <= current_fin_seg + 0.1:
                    current_merged_corte['fin'] = from_seconds(max(current_fin_seg, to_seconds(next_corte['fin'])))
                else:
                    cortes_fusionados.append(current_merged_corte)
                    current_merged_corte = next_corte
            cortes_fusionados.append(current_merged_corte)
        print("\n--- Segmentos a ELIMINAR (final, después de fusión) ---")
        print(json.dumps(cortes_fusionados, indent=2, ensure_ascii=False))
        print("--------------------------------------------------\n")

        # 2. Crear lista de clips "buenos" (Lógica sin cambios)
        clips_buenos = []
        current_time_seg = 0.0
        for corte in cortes_fusionados:
            inicio_corte_seg = to_seconds(corte['inicio'])
            fin_corte_seg = to_seconds(corte['fin'])
            if fin_corte_seg > video_duration: fin_corte_seg = video_duration
            if inicio_corte_seg > current_time_seg:
                if (inicio_corte_seg - current_time_seg) > 0.1:
                    print(f"Manteniendo clip: {from_seconds(current_time_seg)} a {from_seconds(inicio_corte_seg)}")
                    clips_buenos.append(video.subclip(current_time_seg, inicio_corte_seg))
            current_time_seg = max(current_time_seg, fin_corte_seg)
        if (video_duration - current_time_seg) > 0.1:
            print(f"Manteniendo clip final: {from_seconds(current_time_seg)} a {from_seconds(video_duration)}")
            clips_buenos.append(video.subclip(current_time_seg, video_duration))

        # 4. Concatenar (Lógica sin cambios)
        if not clips_buenos:
            print("Advertencia: La lógica de cortes resultó en un video vacío.")
            video.close()
            return None
        print(f"\nEnsamblando video final a partir de {len(clips_buenos)} clips...")
        final_video = concatenate_videoclips(clips_buenos)
        final_video.write_videofile(video_final_path, codec="libx264", audio_codec="aac", logger=None)

        # 6. Limpieza (Lógica sin cambios)
        final_video.close()
        for clip in clips_buenos: clip.close()
        video.close()
        return video_final_path # ¡Devuelve la ruta!
    except Exception as e:
        print(f"Ha ocurrido un error al ensamblar el video: {e}", file=sys.stderr)
        return None
    finally:
        # La limpieza del video original temporal sigue siendo válida
        if os.path.exists(video_original_path):
            os.remove(video_original_path)

# --- FUNCIÓN DE COMPOSICIÓN DE EMOJIS (SIN CAMBIOS) ---
def crear_video_con_emojis(video_base_path, video_final_path, lista_emojis, secciones_para_eliminar):
    """
    Toma el video base cortado y le superpone los emojis.
    """
    print(f"Iniciando Etapa 2: Superposición de Emojis en {video_base_path}...")
    try:
        # 1. Descargar la fuente de emojis
        font_path = descargar_fuente_emoji()
        if not font_path:
            raise Exception("No se pudo descargar la fuente de emojis. Abortando.")

        # 2. Cargar el video base (cortado)
        video_base = VideoFileClip(video_base_path)

        # 3. Preparar el "mapeador de timestamps"
        # Esto convierte el tiempo original al nuevo tiempo (después de los cortes)
        cortes_procesados = sorted(
            [{'inicio': to_seconds(c['inicio']), 'fin': to_seconds(c['fin'])} for c in secciones_para_eliminar],
            key=lambda x: x['inicio']
        )

        def calcular_nuevo_timestamp(tiempo_original_seg):
            tiempo_a_restar = 0.0
            for corte in cortes_procesados:
                if tiempo_original_seg > corte['fin']:
                    tiempo_a_restar += (corte['fin'] - corte['inicio'])
                elif tiempo_original_seg > corte['inicio']:
                    # El tiempo está DENTRO de un corte, ajustar al inicio del corte
                    tiempo_a_restar += (tiempo_original_seg - corte['inicio'])
                    break
            return max(0, tiempo_original_seg - tiempo_a_restar)

        # 4. Crear los clips de texto para cada emoji
        clips_de_emojis = []
        for item in lista_emojis:
            try:
                inicio_original = to_seconds(item['inicio'])
                fin_original = to_seconds(item['fin'])
                duracion = fin_original - inicio_original

                if duracion <= 0.1: # Ignorar emojis de duración inválida
                    continue

                # ¡Calcular el nuevo timestamp!
                nuevo_inicio = calcular_nuevo_timestamp(inicio_original)

                # Asegurarse de que el clip no se salga del video
                if nuevo_inicio + duracion > video_base.duration:
                    duracion = video_base.duration - nuevo_inicio

                if duracion <= 0.1:
                    continue

                print(f"  · Añadiendo emoji {item['emoji']} en {from_seconds(nuevo_inicio)} (Original: {item['inicio']})")

                txt_clip = TextClip(
                    item['emoji'],
                    fontsize=100,
                    font=font_path,
                    color='white' # Fallback, pero NotoColorEmoji tiene su propio color
                )

                txt_clip = txt_clip.set_position(('center', 0.7), relative=True) # Posición al 70% hacia abajo
                txt_clip = txt_clip.set_start(nuevo_inicio)
                txt_clip = txt_clip.set_duration(duracion)
                txt_clip = txt_clip.fadein(0.2).fadeout(0.2) # Efecto suave

                clips_de_emojis.append(txt_clip)

            except Exception as e:
                print(f"Error procesando emoji '{item}': {e}", file=sys.stderr)

        if not clips_de_emojis:
            print("No se generaron emojis. El video final es el video cortado.")
            video_base.close()
            # Renombrar el archivo cortado al nombre final
            if os.path.exists(video_base_path) and not os.path.exists(video_final_path):
                os.rename(video_base_path, video_final_path)
            return video_final_path

        # 5. Componer el video final
        print(f"Componiendo video base con {len(clips_de_emojis)} emojis...")
        video_final_con_emojis = CompositeVideoClip([video_base] + clips_de_emojis)

        video_final_con_emojis.write_videofile(
            video_final_path,
            codec="libx264",
            audio_codec="aac",
            logger=None
        )

        print(f"\n¡Proceso completado! El video con emojis está en '{video_final_path}'.")

        # Limpieza
        video_final_con_emojis.close()
        video_base.close()
        for clip in clips_de_emojis:
            clip.close()

        # <-- ¡MODIFICADO! Limpieza del video cortado intermedio
        if os.path.exists(video_base_path):
             os.remove(video_base_path)
        # --- FIN MODIFICACIÓN ---

        return video_final_path

    except Exception as e:
        print(f"Ha ocurrido un error al crear el video con emojis: {e}", file=sys.stderr)
        return None


# --- Configuración Principal (Orquestador ACTUALIZADO) ---
if __name__ == "__main__":

    # --- ¡EDITA ESTAS LÍNEAS! ---
    # Debes proporcionar la ruta a TUS archivos locales
    RUTA_VIDEO_LOCAL = "to_edit/video.mp4"
    RUTA_SUBTITULOS_LOCAL = "to_edit/subs.vtt"
    # -----------------------------------

    API_KEY = os.getenv("API_KEY")  # ¡CUIDADO CON LA API KEY!

    if "TU_API_KEY_AQUI" in API_KEY: # <-- ¡MODIFICADO! Chequeo simplificado
        print("Error: Estás usando la clave de API de ejemplo.", file=sys.stderr)
        print("Por favor, edita el script y reemplaza la clave de ejemplo", file=sys.stderr)
        sys.exit(1)

    # --- ¡NUEVO CHEQUEO DE ARCHIVOS! ---
    if "AQUI_VA_LA_RUTA" in RUTA_VIDEO_LOCAL or not os.path.exists(RUTA_VIDEO_LOCAL):
        print(f"Error: El archivo de video no se encuentra en '{RUTA_VIDEO_LOCAL}'", file=sys.stderr)
        print("Por favor, edita la variable RUTA_VIDEO_LOCAL en el script.", file=sys.stderr)
        sys.exit(1)
    if "AQUI_VA_LA_RUTA" in RUTA_SUBTITULOS_LOCAL or not os.path.exists(RUTA_SUBTITULOS_LOCAL):
        print(f"Error: El archivo de subtítulos no se encuentra en '{RUTA_SUBTITULOS_LOCAL}'", file=sys.stderr)
        print("Por favor, edita la variable RUTA_SUBTITULOS_LOCAL en el script.", file=sys.stderr)
        sys.exit(1)
    # ---------------------------------

    # URL_VIDEO = "https://www.youtube.com/watch?v=YMsUEDTwdSQ" # <-- ¡Eliminada!

    try:
        # ETAPA 0: OBTENER DATOS (Modificado)
        transcripcion, subtitulos_obj = leer_subtitulos_locales(RUTA_SUBTITULOS_LOCAL) # <-- ¡MODIFICADO!
        if not transcripcion or not subtitulos_obj:
            raise Exception("No se pudo obtener la transcripción o los subtítulos del archivo local.")

        # ETAPA 1: LÓGICA DE CORTE (Sin cambios)
        cortes_ia = obtener_cortes_para_eliminar(API_KEY, transcripcion, duracion_minima_seg=0.1)
        cortes_silencio = identificar_silencios_largos(subtitulos_obj, umbral_segundos=1.5)
        todos_los_cortes = cortes_ia + cortes_silencio

        if not todos_los_cortes:
            print("\nIA y análisis de silencios no encontraron nada que eliminar. ¡Video limpio!")
        else:
            print("\n--- Segmentos totales a eliminar (antes de fusión) ---")
            print(json.dumps(todos_los_cortes, indent=2, ensure_ascii=False))
            print("--------------------------------------------------\n")

        # ETAPA 1.5: ENSAMBLAR VIDEO BASE (Modificado)
        ruta_video_cortado = ensamblar_video_editado(RUTA_VIDEO_LOCAL, todos_los_cortes) # <-- ¡MODIFICADO!

        if not ruta_video_cortado:
            raise Exception("Falló la etapa de corte de video.")

        print(f"Etapa 1 completada. Video cortado guardado en: {ruta_video_cortado}")

        # ETAPA 2: LÓGICA DE EMOJIS (Sin cambios)
        lista_emojis = obtener_emojis_para_subtitulos(API_KEY, subtitulos_obj, todos_los_cortes)

        if not lista_emojis:
            print("IA no sugirió emojis. El proceso ha finalizado.")
            # <-- ¡MODIFICADO! Asegurarse de que el archivo final tenga el nombre correcto
            if os.path.exists(ruta_video_cortado) and not os.path.exists("video_final_con_emojis.mp4"):
                os.rename(ruta_video_cortado, "video_final_con_emojis.mp4")
                print(f"Video final (solo cortado) guardado como: video_final_con_emojis.mp4")
            sys.exit(0)
        else:
            print("\n--- Emojis sugeridos por la IA ---")
            print(json.dumps(lista_emojis, indent=2, ensure_ascii=False))
            print("------------------------------------\n")

        # ETAPA 2.5: ENSAMBLAR VIDEO FINAL CON EMOJIS (Sin cambios)
        ruta_video_final_con_emojis = "video_final_con_emojis.mp4"
        crear_video_con_emojis(
            ruta_video_cortado,
            ruta_video_final_con_emojis,
            lista_emojis,
            todos_los_cortes
        )

    except Exception as e:
        print(f"El proceso principal falló: {e}", file=sys.stderr)
        # Limpieza final en caso de error
        if os.path.exists("video_original_para_editar.mp4"):
            os.remove("video_original_para_editar.mp4")
        if os.path.exists("video_final_editado.mp4"):
            os.remove("video_final_editado.mp4")
        sys.exit(1)

ModuleNotFoundError: No module named 'moviepy.editor'

# Shorts

In [ ]:
import os
import subprocess
import webvtt
import google.generativeai as genai
import json
import sys
from moviepy.editor import VideoFileClip

def to_seconds(time_str):
    """Convierte un string de tiempo 'HH:MM:SS' o 'MM:SS' a segundos."""
    try:
        parts = list(map(int, time_str.split(':')))
        if len(parts) == 3:  # HH:MM:SS
            h, m, s = parts
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:  # MM:SS
            m, s = parts
            return m * 60 + s
        else:
            raise ValueError("Formato de tiempo no válido")
    except ValueError as e:
        print(f"Error al convertir tiempo '{time_str}': {e}", file=sys.stderr)
        return 0

def extraer_subtitulos(video_url):
    """
    Descarga solo los subtítulos usando yt-dlp y los devuelve
    como un texto formateado Y como un objeto webvtt.
    """
    vtt_path = None
    captions = None  # <--- NUEVO
    transcript = ""  # <--- NUEVO

    try:
        print(f"Descargando subtítulos para: {video_url}...")
        comando_subs = [
            'yt-dlp',
            '--write-auto-subs',
            '--sub-langs', 'es',
            '--convert-subs', 'vtt',
            '--skip-download',
            '-o', 'subs_temp.%(ext)s',
            video_url
        ]

        subprocess.run(comando_subs, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        found_files = [f for f in os.listdir('.') if f.startswith('subs_temp.') and f.endswith('.vtt')]
        if found_files:
            vtt_path = found_files[0]
        else:
             raise FileNotFoundError("No se pudo encontrar el archivo .vtt descargado.")

        print("Subtítulos descargados. Leyendo y formateando...")

        captions = webvtt.read(vtt_path)  # <--- MODIFICADO (asignando a la variable externa)
        for caption in captions:
            secs = int(caption.start_in_seconds)
            h = secs // 3600
            m = (secs % 3600) // 60
            s = secs % 60
            timestamp = f"{h:02}:{m:02}:{s:02}"
            transcript += f"[{timestamp}] {caption.text.strip().replace(chr(10), ' ')}\n"

        return transcript, captions  # <--- MODIFICADO (devuelve ambos)

    except Exception as e:
        print(f"Error extrayendo subtítulos: {e}", file=sys.stderr)
        return None, None  # <--- MODIFICADO (devuelve None para ambos)
    finally:
        if vtt_path and os.path.exists(vtt_path):
            os.remove(vtt_path)

def obtener_clips_con_gemini(api_key, transcript):
    """
    Envía la transcripción a Gemini y le pide que identifique
    secciones para shorts, devolviendo un JSON.
    """
    print("Conectando con Gemini para analizar momentos clave...")
    try:
        genai.configure(api_key=api_key)

        # Usamos 1.5-flash, es rápido y muy capaz
        model = genai.GenerativeModel('gemini-2.0-flash')

        prompt = f"""
        Eres un experto productor de video y creador de contenido viral
        para YouTube Shorts.

        Tu tarea es analizar la siguiente transcripción de un video
        (con marcas de tiempo [HH:MM:SS]) e identificar los 3 a 5
        momentos más impactantes, educativos o "virales" que serían
        perfectos para YouTube Shorts.

        Reglas para los clips:
        1. Cada clip debe tener una duración ideal de 30 a 55 segundos.
        2. Los clips no deben superponerse.
        3. Busca "ganchos" (preguntas, declaraciones impactantes),
           resúmenes de puntos clave o conclusiones claras.
        4. **REGLA CRÍTICA DE COMPLETITUD:** El timestamp de 'fin' DEBE
           corresponder al final de una oración o una idea completa.
           No cortes al orador a mitad de frase. Es PREFERIBLE que el
           clip dure unos segundos más (hasta 59 segundos) para
           asegurar que la idea se concluya, en lugar de cortarlo
           abruptamente.

        Tu respuesta DEBE ser únicamente un objeto JSON, sin ningún
        otro texto, markdown o explicación.

        El formato JSON debe ser una lista de objetos, donde cada
        objeto tiene tres claves:
        - "nombre": Un título corto y pegadizo para el clip (usa guiones bajos, sin espacios).
        - "inicio": El timestamp 'HH:MM:SS' de inicio.
        - "fin": El timestamp 'HH:MM:SS' de fin.

        Ejemplo de respuesta:
        [
          {{"nombre": "Error_Comun_al_Estudiar", "inicio": "00:03:05", "fin": "00:03:52"}},
          {{"nombre": "Reseña_Libro_Saunders", "inicio": "00:08:05", "fin": "00:08:47"}}
        ]

        Aquí está la transcripción:
        ---
        {transcript}
        ---
        """

        response = model.generate_content(prompt)

        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")

        print("Análisis de IA completado. Parseando secciones...")
        secciones = json.loads(cleaned_response)
        return secciones

    except Exception as e:
        print(f"Error al contactar con Gemini o parsear JSON: {e}", file=sys.stderr)
        if "404" in str(e) and "is not found" in str(e):
            print("\n--- ¡PROBLEMA DE API! ---", file=sys.stderr)
            print("Error 404: El modelo no se encuentra.", file=sys.stderr)
            print("Esto casi siempre significa que tu API KEY tiene un problema.", file=sys.stderr)
            print("1. Tu clave de API fue (o será) desactivada por seguridad.", file=sys.stderr)
            print("2. Debes habilitar la API 'Generative Language API' en tu proyecto de Google Cloud.", file=sys.stderr)
        return None

def cortar_video_clips(video_url, secciones, captions_obj): # <--- MODIFICADO (nuevo argumento)
    """
    Descarga el video completo, corta las secciones de la IA
    y guarda un .txt con el subtítulo de cada una.
    """
    video_original_path = "video_original.mp4"

    try:
        print(f"Descargando video completo con yt-dlp: {video_url}")
        comando_descarga = [
            'yt-dlp',
            '-f', 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
            '-o', video_original_path,
            '--force-overwrite',
            video_url
        ]
        subprocess.run(comando_descarga, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print("Video descargado exitosamente.")

        output_folder = "shorts_generados_por_ia"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for seccion in secciones:
            nombre = seccion['nombre']
            inicio_str = seccion['inicio']
            fin_str = seccion['fin']

            video = None
            clip_original = None
            clip_recortado = None

            try:
                video = VideoFileClip(video_original_path)

                inicio_seg = to_seconds(inicio_str)
                fin_seg = to_seconds(fin_str)

                if fin_seg > video.duration:
                    print(f"Advertencia: Tiempo final '{fin_str}' excede duración. Cortando al final del video.")
                    fin_seg = video.duration

                if inicio_seg >= fin_seg:
                    print(f"Advertencia: Tiempo de inicio '{inicio_str}' es mayor o igual al de fin '{fin_str}'. Saltando clip.")
                    continue

                print(f"Creando clip IA: {nombre} ({inicio_str} a {fin_str})...")

                clip_original = video.subclip(inicio_seg, fin_seg)

                (w, h) = clip_original.size
                target_width = h * 9 / 16
                x_center = w / 2
                x1 = x_center - (target_width / 2)
                x2 = x_center + (target_width / 2)
                clip_recortado = clip_original.crop(x1=x1, y1=0, x2=x2, y2=h)

                output_path = os.path.join(output_folder, f"{nombre}.mp4")
                clip_recortado.write_videofile(output_path,
                                              codec="libx264",
                                              audio_codec="aac",
                                              logger=None)

                print(f"Clip '{nombre}' creado exitosamente.")

                # --- ¡SECCIÓN NUEVA PARA GUARDAR EL TXT! ---
                texto_del_short = []
                if captions_obj:
                    for caption in captions_obj:
                        # Comprobar si el subtítulo se solapa con el clip
                        if caption.start_in_seconds < fin_seg and caption.end_in_seconds > inicio_seg:
                            clean_text = caption.text.strip().replace('\n', ' ')
                            texto_del_short.append(clean_text)

                # Unir las líneas en un solo bloque de texto
                texto_completo = "\n".join(texto_del_short)

                # Definir la ruta del archivo .txt
                txt_output_path = os.path.join(output_folder, f"{nombre}.txt")

                try:
                    with open(txt_output_path, 'w', encoding='utf-8') as f:
                        f.write(texto_completo)
                    print(f"Texto del clip '{nombre}' guardado en .txt.")
                except Exception as e:
                    print(f"Advertencia: No se pudo guardar el .txt para '{nombre}': {e}")
                # --- FIN DE LA SECCIÓN NUEVA ---

            finally:
                if clip_recortado: clip_recortado.close()
                if clip_original: clip_original.close()
                if video: video.close()

        os.remove(video_original_path)
        print(f"\n¡Proceso completado! Los shorts y .txt están en la carpeta '{output_folder}'.")

    except subprocess.CalledProcessError as e:
        print(f"Error al descargar con yt-dlp: {e}", file=sys.stderr)
    except Exception as e:
        print(f"Ha ocurrido un error al cortar los clips: {e}", file=sys.stderr)
        if os.path.exists(video_original_path):
            os.remove(video_original_path)

# --- Configuración Principal (Orquestador) ---

if __name__ == "__main__":

    # --- CORRECCIÓN DE SEGURIDAD OBLIGATORIA ---
    # He revertido tu cambio. NUNCA escribas tu clave aquí.
    # El script fallará si no la configuras como variable de entorno.
    API_KEY = os.getenv("API_KEY")

    if not API_KEY:
        print("Error: No se encontró la variable de entorno 'GEMINI_API_KEY'.", file=sys.stderr)
        print("Por favor, configúrala antes de ejecutar el script:", file=sys.stderr)
        print("  En Linux/macOS: export GEMINI_API_KEY=\"TU_NUEVA_CLAVE_SECRETA\"", file=sys.stderr)
        print("  En Windows CMD: set GEMINI_API_KEY=\"TU_NUEVA_CLAVE_SECRETA\"", file=sys.stderr)
        sys.exit(1)

    URL_VIDEO = "https://www.youtube.com/watch?v=YMsUEDTwdSQ"

    try:
        # <--- MODIFICADO (ahora recibe dos variables)
        transcripcion, subtitulos_obj = extraer_subtitulos(URL_VIDEO)

        if not transcripcion or not subtitulos_obj: # <--- MODIFICADO
            raise Exception("No se pudo obtener la transcripción o los subtítulos.")

        secciones_ai = obtener_clips_con_gemini(API_KEY, transcripcion)
        if not secciones_ai:
            raise Exception("Gemini no devolvió secciones válidas.")

        print("\n--- Secciones identificadas por Gemini ---")
        print(json.dumps(secciones_ai, indent=2, ensure_ascii=False))
        print("------------------------------------------\n")

        # <--- MODIFICADO (pasa el objeto de subtítulos a la función)
        cortar_video_clips(URL_VIDEO, secciones_ai, subtitulos_obj)

    except Exception as e:
        print(f"El proceso falló: {e}", file=sys.stderr)
        sys.exit(1)